# Handling imbalanced data in customer churn prediction

Customer churn prediction is to measure why customers are leaving a business. In this notebook we will be looking at customer churn in telecom business. We will build a deep learning model to predict the churn and use precision, recall, f1-score to measure performance of our model. We will then handle imbalance in data using various techniques and improve f1-score

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('customer_churn.csv')

In [3]:
df

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,6840-RESVB,Male,0,Yes,Yes,24,Yes,Yes,DSL,Yes,...,Yes,Yes,Yes,Yes,One year,Yes,Mailed check,84.80,1990.5,No
7039,2234-XADUH,Female,0,Yes,Yes,72,Yes,Yes,Fiber optic,No,...,Yes,No,Yes,Yes,One year,Yes,Credit card (automatic),103.20,7362.9,No
7040,4801-JZAZL,Female,0,Yes,Yes,11,No,No phone service,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.60,346.45,No
7041,8361-LTMKD,Male,1,Yes,No,4,Yes,Yes,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Mailed check,74.40,306.6,Yes


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


In [5]:
df['Churn'].value_counts()

No     5174
Yes    1869
Name: Churn, dtype: int64

In [6]:
xdf = df.copy()

### Dropping Customer ID

In [7]:
xdf.drop('customerID', axis = 1, inplace = True)

In [8]:
## What's the ratio 

5174 / df.shape[0]

0.7346301292063041

In [9]:
1869 / df.shape[0]

0.2653698707936959

The ratio is <b> 73 % </b> - <b> 26% </b>

#### TotalCharges should be 'float'

In [10]:
xdf.TotalCharges.values

array(['29.85', '1889.5', '108.15', ..., '346.45', '306.6', '6844.5'],
      dtype=object)

In [11]:
pd.to_numeric(xdf.TotalCharges, errors = 'coerce').isnull()

0       False
1       False
2       False
3       False
4       False
        ...  
7038    False
7039    False
7040    False
7041    False
7042    False
Name: TotalCharges, Length: 7043, dtype: bool

In [12]:
xdf[pd.to_numeric(xdf.TotalCharges, errors = 'coerce').isnull()]

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
488,Female,0,Yes,Yes,0,No,No phone service,DSL,Yes,No,Yes,Yes,Yes,No,Two year,Yes,Bank transfer (automatic),52.55,,No
753,Male,0,No,Yes,0,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,20.25,,No
936,Female,0,Yes,Yes,0,Yes,No,DSL,Yes,Yes,Yes,No,Yes,Yes,Two year,No,Mailed check,80.85,,No
1082,Male,0,Yes,Yes,0,Yes,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,25.75,,No
1340,Female,0,Yes,Yes,0,No,No phone service,DSL,Yes,Yes,Yes,Yes,Yes,No,Two year,No,Credit card (automatic),56.05,,No
3331,Male,0,Yes,Yes,0,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,19.85,,No
3826,Male,0,Yes,Yes,0,Yes,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,25.35,,No
4380,Female,0,Yes,Yes,0,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,20.00,,No
5218,Male,0,Yes,Yes,0,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,One year,Yes,Mailed check,19.70,,No
6670,Female,0,Yes,Yes,0,Yes,Yes,DSL,No,Yes,Yes,Yes,Yes,No,Two year,No,Mailed check,73.35,,No


In [13]:
xdf.shape

(7043, 20)

In [14]:
xdf.iloc[488].TotalCharges

' '

In [15]:
xdf[xdf.TotalCharges != ' '].shape

(7032, 20)

#### Remove rows with space in TotalCharges

In [16]:
xdf = xdf[xdf.TotalCharges!=' ']
xdf.shape

(7032, 20)

In [17]:
xdf.dtypes

gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object

In [18]:
xdf.TotalCharges = pd.to_numeric(xdf.TotalCharges)

In [19]:
xdf.TotalCharges.values

array([  29.85, 1889.5 ,  108.15, ...,  346.45,  306.6 , 6844.5 ])

In [20]:
xdf[xdf.Churn == 'No']

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,No
3,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
6,Male,0,No,Yes,22,Yes,Yes,Fiber optic,No,Yes,No,No,Yes,No,Month-to-month,Yes,Credit card (automatic),89.10,1949.40,No
7,Female,0,No,No,10,No,No phone service,DSL,Yes,No,No,No,No,No,Month-to-month,No,Mailed check,29.75,301.90,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7037,Female,0,No,No,72,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,Yes,Bank transfer (automatic),21.15,1419.40,No
7038,Male,0,Yes,Yes,24,Yes,Yes,DSL,Yes,No,Yes,Yes,Yes,Yes,One year,Yes,Mailed check,84.80,1990.50,No
7039,Female,0,Yes,Yes,72,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes,One year,Yes,Credit card (automatic),103.20,7362.90,No
7040,Female,0,Yes,Yes,11,No,No phone service,DSL,Yes,No,No,No,No,No,Month-to-month,Yes,Electronic check,29.60,346.45,No


#### Let's print unique values in object columns to see data values

In [21]:
def print_unique_col_values(df):
    for column in df:
        if df[column].dtypes == 'object':
            print(f'{column}: {df[column].unique()}')

In [22]:
print_unique_col_values(xdf)

gender: ['Female' 'Male']
Partner: ['Yes' 'No']
Dependents: ['No' 'Yes']
PhoneService: ['No' 'Yes']
MultipleLines: ['No phone service' 'No' 'Yes']
InternetService: ['DSL' 'Fiber optic' 'No']
OnlineSecurity: ['No' 'Yes' 'No internet service']
OnlineBackup: ['Yes' 'No' 'No internet service']
DeviceProtection: ['No' 'Yes' 'No internet service']
TechSupport: ['No' 'Yes' 'No internet service']
StreamingTV: ['No' 'Yes' 'No internet service']
StreamingMovies: ['No' 'Yes' 'No internet service']
Contract: ['Month-to-month' 'One year' 'Two year']
PaperlessBilling: ['Yes' 'No']
PaymentMethod: ['Electronic check' 'Mailed check' 'Bank transfer (automatic)'
 'Credit card (automatic)']
Churn: ['No' 'Yes']


Some of the columns have no internet service or no phone service, that can be replaced with a simple No

In [23]:
xdf.replace('No internet service','No', inplace = True)
xdf.replace('No phone service','No', inplace = True)

In [24]:
print_unique_col_values(xdf)

gender: ['Female' 'Male']
Partner: ['Yes' 'No']
Dependents: ['No' 'Yes']
PhoneService: ['No' 'Yes']
MultipleLines: ['No' 'Yes']
InternetService: ['DSL' 'Fiber optic' 'No']
OnlineSecurity: ['No' 'Yes']
OnlineBackup: ['Yes' 'No']
DeviceProtection: ['No' 'Yes']
TechSupport: ['No' 'Yes']
StreamingTV: ['No' 'Yes']
StreamingMovies: ['No' 'Yes']
Contract: ['Month-to-month' 'One year' 'Two year']
PaperlessBilling: ['Yes' 'No']
PaymentMethod: ['Electronic check' 'Mailed check' 'Bank transfer (automatic)'
 'Credit card (automatic)']
Churn: ['No' 'Yes']


#### Convert Yes and No to 1 or 0

In [25]:
yes_no_columns = ['Partner','Dependents','PhoneService','MultipleLines','OnlineSecurity','OnlineBackup','DeviceProtection','TechSupport','StreamingTV','StreamingMovies','PaperlessBilling','Churn']

for col in yes_no_columns:
    xdf[col].replace({'Yes':1 ,'No': 0}, inplace = True)

In [26]:
for col in xdf:
    print(f'{col}:{xdf[col].unique()}')

gender:['Female' 'Male']
SeniorCitizen:[0 1]
Partner:[1 0]
Dependents:[0 1]
tenure:[ 1 34  2 45  8 22 10 28 62 13 16 58 49 25 69 52 71 21 12 30 47 72 17 27
  5 46 11 70 63 43 15 60 18 66  9  3 31 50 64 56  7 42 35 48 29 65 38 68
 32 55 37 36 41  6  4 33 67 23 57 61 14 20 53 40 59 24 44 19 54 51 26 39]
PhoneService:[0 1]
MultipleLines:[0 1]
InternetService:['DSL' 'Fiber optic' 'No']
OnlineSecurity:[0 1]
OnlineBackup:[1 0]
DeviceProtection:[0 1]
TechSupport:[0 1]
StreamingTV:[0 1]
StreamingMovies:[0 1]
Contract:['Month-to-month' 'One year' 'Two year']
PaperlessBilling:[1 0]
PaymentMethod:['Electronic check' 'Mailed check' 'Bank transfer (automatic)'
 'Credit card (automatic)']
MonthlyCharges:[29.85 56.95 53.85 ... 63.1  44.2  78.7 ]
TotalCharges:[  29.85 1889.5   108.15 ...  346.45  306.6  6844.5 ]
Churn:[0 1]


In [27]:
xdf['gender'].replace({'Female':1, 'Male':0}, inplace = True)

In [28]:
xdf.gender.unique()

array([1, 0], dtype=int64)

#### One hot encoding for categorical columns

In [29]:
xxdf = pd.get_dummies(data = xdf, columns = ['InternetService','Contract','PaymentMethod'])

xxdf.columns

Index(['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'tenure',
       'PhoneService', 'MultipleLines', 'OnlineSecurity', 'OnlineBackup',
       'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies',
       'PaperlessBilling', 'MonthlyCharges', 'TotalCharges', 'Churn',
       'InternetService_DSL', 'InternetService_Fiber optic',
       'InternetService_No', 'Contract_Month-to-month', 'Contract_One year',
       'Contract_Two year', 'PaymentMethod_Bank transfer (automatic)',
       'PaymentMethod_Credit card (automatic)',
       'PaymentMethod_Electronic check', 'PaymentMethod_Mailed check'],
      dtype='object')

In [30]:
xxdf

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,OnlineSecurity,OnlineBackup,DeviceProtection,...,InternetService_DSL,InternetService_Fiber optic,InternetService_No,Contract_Month-to-month,Contract_One year,Contract_Two year,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check
0,1,0,1,0,1,0,0,0,1,0,...,1,0,0,1,0,0,0,0,1,0
1,0,0,0,0,34,1,0,1,0,1,...,1,0,0,0,1,0,0,0,0,1
2,0,0,0,0,2,1,0,1,1,0,...,1,0,0,1,0,0,0,0,0,1
3,0,0,0,0,45,0,0,1,0,1,...,1,0,0,0,1,0,1,0,0,0
4,1,0,0,0,2,1,0,0,0,0,...,0,1,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,0,0,1,1,24,1,1,1,0,1,...,1,0,0,0,1,0,0,0,0,1
7039,1,0,1,1,72,1,1,0,1,1,...,0,1,0,0,1,0,0,1,0,0
7040,1,0,1,1,11,0,0,1,0,0,...,1,0,0,1,0,0,0,0,1,0
7041,0,1,1,0,4,1,1,0,0,0,...,0,1,0,1,0,0,0,0,0,1


In [31]:
xxdf.dtypes

gender                                       int64
SeniorCitizen                                int64
Partner                                      int64
Dependents                                   int64
tenure                                       int64
PhoneService                                 int64
MultipleLines                                int64
OnlineSecurity                               int64
OnlineBackup                                 int64
DeviceProtection                             int64
TechSupport                                  int64
StreamingTV                                  int64
StreamingMovies                              int64
PaperlessBilling                             int64
MonthlyCharges                             float64
TotalCharges                               float64
Churn                                        int64
InternetService_DSL                          uint8
InternetService_Fiber optic                  uint8
InternetService_No             

In [32]:
cols_to_scale = ['tenure','MonthlyCharges','TotalCharges']

from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

xxdf[cols_to_scale] = scaler.fit_transform(xxdf[cols_to_scale])

In [33]:
for col in xxdf:
    print(f'{col}: {xxdf[col].unique()}')

gender: [1 0]
SeniorCitizen: [0 1]
Partner: [1 0]
Dependents: [0 1]
tenure: [0.         0.46478873 0.01408451 0.61971831 0.09859155 0.29577465
 0.12676056 0.38028169 0.85915493 0.16901408 0.21126761 0.8028169
 0.67605634 0.33802817 0.95774648 0.71830986 0.98591549 0.28169014
 0.15492958 0.4084507  0.64788732 1.         0.22535211 0.36619718
 0.05633803 0.63380282 0.14084507 0.97183099 0.87323944 0.5915493
 0.1971831  0.83098592 0.23943662 0.91549296 0.11267606 0.02816901
 0.42253521 0.69014085 0.88732394 0.77464789 0.08450704 0.57746479
 0.47887324 0.66197183 0.3943662  0.90140845 0.52112676 0.94366197
 0.43661972 0.76056338 0.50704225 0.49295775 0.56338028 0.07042254
 0.04225352 0.45070423 0.92957746 0.30985915 0.78873239 0.84507042
 0.18309859 0.26760563 0.73239437 0.54929577 0.81690141 0.32394366
 0.6056338  0.25352113 0.74647887 0.70422535 0.35211268 0.53521127]
PhoneService: [0 1]
MultipleLines: [0 1]
OnlineSecurity: [0 1]
OnlineBackup: [1 0]
DeviceProtection: [0 1]
TechSupport: [

### Train test split

In [34]:
X = xxdf.drop('Churn', axis = 1)
y = xxdf.Churn.astype(np.float32)


In [35]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state =  15, stratify = y)

In [36]:
y_train.value_counts()

0.0    4130
1.0    1495
Name: Churn, dtype: int64

In [37]:
y.value_counts()

0.0    5163
1.0    1869
Name: Churn, dtype: int64

### Build a model (ANN) in tensorflow/keras

In [38]:
from tensorflow_addons import losses

In [39]:
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import confusion_matrix, classification_report

In [40]:
def ANN(X_train, y_train, X_test, y_test, loss, weights):
    model = keras.Sequential([
        keras.layers.Dense(26, input_dim = 26, activation = 'relu'),
        keras.layers.Dense(15, activation = 'relu'),
        keras.layers.Dense(1, activation = 'sigmoid')
    ])
    
    model.compile(optimizer = 'adam', loss = loss, metrics = ['accuracy'])
    
    if weights == -1:
        model.fit(X_train, y_train, epochs = 100)
    else:
        model.fit(X_train, y_train, epochs = 100, class_weight = weights)
        
    print(model.evaluate(X_test, y_test))
    
    y_preds = model.predict(X_test)
    y_preds = np.round(y_preds)
    
    print("Classification Report: \n", classification_report(y_test, y_preds))
    
    return y_preds
    

In [41]:
y_preds = ANN(X_train, y_train, X_test, y_test,'binary_crossentropy', -1)

Epoch 1/100
176/176 [==============================] - 1s 1ms/step - loss: 0.5439 - accuracy: 0.7370
Epoch 2/100
176/176 [==============================] - 0s 1ms/step - loss: 0.4313 - accuracy: 0.7904
Epoch 3/100
176/176 [==============================] - 0s 1ms/step - loss: 0.4210 - accuracy: 0.7953
Epoch 4/100
176/176 [==============================] - 0s 1ms/step - loss: 0.4186 - accuracy: 0.7993
Epoch 5/100
176/176 [==============================] - 0s 1ms/step - loss: 0.4168 - accuracy: 0.8050
Epoch 6/100
176/176 [==============================] - 0s 1ms/step - loss: 0.4089 - accuracy: 0.8048
Epoch 7/100
176/176 [==============================] - 0s 1ms/step - loss: 0.4094 - accuracy: 0.8015
Epoch 8/100
176/176 [==============================] - 0s 1ms/step - loss: 0.4102 - accuracy: 0.8092
Epoch 9/100
176/176 [==============================] - 0s 1ms/step - loss: 0.4046 - accuracy: 0.7997
Epoch 10/100
176/176 [==============================] - 0s 1ms/step - loss: 0.4143 - accura

## Mitigating Skewdness of Data

### Method 1: Undersampling

In [68]:
xxdf['Churn'].value_counts()

0    5163
1    1869
Name: Churn, dtype: int64

In [67]:
# Let's shuffle the data

under_samp = xxdf.sample(frac = 1)

In [71]:
# ratio of 'Churn' is 1869
# nchrn_df = no churn
# chr_df = Churn

nchrn_df = under_samp.loc[under_samp['Churn'] == 0][:1869]
churn = under_samp.loc[under_samp['Churn'] == 1]

under_sampled_df = pd.concat([nchrn_df, churn])

# Shuffle dataframe rows

under_sampled_df = under_sampled_df.sample(frac = 1, random_state = 42)

In [74]:
X = under_sampled_df.drop('Churn', axis = 1)
y = under_sampled_df['Churn']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 42, stratify = y)

In [75]:
y_train.value_counts()

0    1495
1    1495
Name: Churn, dtype: int64

In [76]:
# 

y_preds = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy', -1)

Epoch 1/100
94/94 [==============================] - 1s 611us/step - loss: 0.6523 - accuracy: 0.6364
Epoch 2/100
94/94 [==============================] - 0s 557us/step - loss: 0.5239 - accuracy: 0.7579
Epoch 3/100
94/94 [==============================] - 0s 525us/step - loss: 0.5069 - accuracy: 0.7560
Epoch 4/100
94/94 [==============================] - 0s 515us/step - loss: 0.4844 - accuracy: 0.7596
Epoch 5/100
94/94 [==============================] - 0s 527us/step - loss: 0.4913 - accuracy: 0.7581
Epoch 6/100
94/94 [==============================] - 0s 515us/step - loss: 0.4786 - accuracy: 0.7624
Epoch 7/100
94/94 [==============================] - 0s 563us/step - loss: 0.4781 - accuracy: 0.7690
Epoch 8/100
94/94 [==============================] - 0s 618us/step - loss: 0.4858 - accuracy: 0.7589
Epoch 9/100
94/94 [==============================] - 0s 679us/step - loss: 0.4585 - accuracy: 0.7850
Epoch 10/100
94/94 [==============================] - 0s 600us/step - loss: 0.4634 - accura

94/94 [==============================] - 0s 675us/step - loss: 0.4168 - accuracy: 0.8123
Epoch 82/100
94/94 [==============================] - 0s 617us/step - loss: 0.4100 - accuracy: 0.8121
Epoch 83/100
94/94 [==============================] - 0s 606us/step - loss: 0.4012 - accuracy: 0.8118
Epoch 84/100
94/94 [==============================] - 0s 495us/step - loss: 0.4100 - accuracy: 0.7995
Epoch 85/100
94/94 [==============================] - 0s 504us/step - loss: 0.3945 - accuracy: 0.8189
Epoch 86/100
94/94 [==============================] - 0s 567us/step - loss: 0.4137 - accuracy: 0.7991
Epoch 87/100
94/94 [==============================] - 0s 645us/step - loss: 0.3968 - accuracy: 0.8153
Epoch 88/100
94/94 [==============================] - 0s 646us/step - loss: 0.3887 - accuracy: 0.8220
Epoch 89/100
94/94 [==============================] - 0s 664us/step - loss: 0.4062 - accuracy: 0.8065
Epoch 90/100
94/94 [==============================] - 0s 638us/step - loss: 0.3899 - accuracy: 

# OverSampling

In [85]:
count_class_0, count_class_1 = xdf.Churn.value_counts()

(count_class_0, count_class_1)

(5163, 1869)

In [83]:
# Divide by class

df_class_0 = xxdf[xxdf['Churn'] == 0]
df_class_1 = xxdf[xxdf['Churn'] == 1]

In [86]:
# Oversample 1-class and concat the DataFrmaes of both classes

oversample_1_df = df_class_1.sample(count_class_0, replace = True)

df_oversampled = pd.concat([df_class_0, oversample_1_df], axis = 0)


In [87]:
print("Random Over-Sampling:")
print(df_oversampled.Churn.value_counts())

Random Over-Sampling:
0    5163
1    5163
Name: Churn, dtype: int64


In [88]:
X = df_oversampled.drop('Churn', axis = 1)
y = df_oversampled['Churn']

In [89]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42, stratify = y)

In [90]:
# Number of classes in training data

y_train.value_counts()

0    4130
1    4130
Name: Churn, dtype: int64

In [93]:
loss = keras.losses.BinaryCrossentropy()
weigths = -1

y_preds = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy', -1)

Epoch 1/100
259/259 [==============================] - 0s 581us/step - loss: 0.6341 - accuracy: 0.6147
Epoch 2/100
259/259 [==============================] - 0s 551us/step - loss: 0.5022 - accuracy: 0.7602
Epoch 3/100
259/259 [==============================] - 0s 639us/step - loss: 0.4947 - accuracy: 0.7577
Epoch 4/100
259/259 [==============================] - 0s 702us/step - loss: 0.4927 - accuracy: 0.7614
Epoch 5/100
259/259 [==============================] - 0s 549us/step - loss: 0.4848 - accuracy: 0.7602
Epoch 6/100
259/259 [==============================] - 0s 686us/step - loss: 0.4785 - accuracy: 0.7709
Epoch 7/100
259/259 [==============================] - 0s 660us/step - loss: 0.4654 - accuracy: 0.7811
Epoch 8/100
259/259 [==============================] - 0s 528us/step - loss: 0.4679 - accuracy: 0.7773
Epoch 9/100
259/259 [==============================] - 0s 557us/step - loss: 0.4649 - accuracy: 0.7807
Epoch 10/100
259/259 [==============================] - 0s 688us/step - l

259/259 [==============================] - 0s 683us/step - loss: 0.3838 - accuracy: 0.8227
Epoch 80/100
259/259 [==============================] - 0s 612us/step - loss: 0.3763 - accuracy: 0.83620s - loss: 0.3661 - accuracy: 
Epoch 81/100
259/259 [==============================] - 0s 524us/step - loss: 0.3757 - accuracy: 0.8309
Epoch 82/100
259/259 [==============================] - 0s 527us/step - loss: 0.3679 - accuracy: 0.8341
Epoch 83/100
259/259 [==============================] - 0s 521us/step - loss: 0.3747 - accuracy: 0.8316
Epoch 84/100
259/259 [==============================] - 0s 537us/step - loss: 0.3642 - accuracy: 0.8395
Epoch 85/100
259/259 [==============================] - 0s 519us/step - loss: 0.3731 - accuracy: 0.8337
Epoch 86/100
259/259 [==============================] - 0s 552us/step - loss: 0.3772 - accuracy: 0.8354
Epoch 87/100
259/259 [==============================] - 0s 510us/step - loss: 0.3747 - accuracy: 0.8357
Epoch 88/100
259/259 [=========================

# SMOTE


In [94]:
!pip install imbalanced-learn

In [95]:
X = xxdf.drop('Churn', axis = 1)
y = xxdf['Churn']

In [101]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(sampling_strategy = 'minority')
X_sm, y_sm = smote.fit_resample(X,y)

In [102]:
y_sm.value_counts()

0    5163
1    5163
Name: Churn, dtype: int64

In [103]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_sm, y_sm, test_size = 0.2, random_state = 42, stratify = y_sm)

In [104]:
# Number of classes in training data

y_train.value_counts()

0    4130
1    4130
Name: Churn, dtype: int64

In [105]:
y_preds = ANN(X_train, y_train, X_test, y_test ,'binary_crossentropy', -1)

Epoch 1/100
259/259 [==============================] - 0s 561us/step - loss: 0.6001 - accuracy: 0.6812
Epoch 2/100
259/259 [==============================] - 0s 546us/step - loss: 0.4598 - accuracy: 0.7901
Epoch 3/100
259/259 [==============================] - 0s 649us/step - loss: 0.4487 - accuracy: 0.7905
Epoch 4/100
259/259 [==============================] - 0s 550us/step - loss: 0.4511 - accuracy: 0.7870
Epoch 5/100
259/259 [==============================] - 0s 695us/step - loss: 0.4425 - accuracy: 0.7897
Epoch 6/100
259/259 [==============================] - 0s 569us/step - loss: 0.4433 - accuracy: 0.7895
Epoch 7/100
259/259 [==============================] - 0s 576us/step - loss: 0.4343 - accuracy: 0.7935
Epoch 8/100
259/259 [==============================] - 0s 716us/step - loss: 0.4374 - accuracy: 0.7967
Epoch 9/100
259/259 [==============================] - 0s 573us/step - loss: 0.4423 - accuracy: 0.7923
Epoch 10/100
259/259 [==============================] - 0s 579us/step - l

259/259 [==============================] - 0s 528us/step - loss: 0.3488 - accuracy: 0.8480
Epoch 80/100
259/259 [==============================] - 0s 677us/step - loss: 0.3372 - accuracy: 0.8473
Epoch 81/100
259/259 [==============================] - 0s 543us/step - loss: 0.3431 - accuracy: 0.8518
Epoch 82/100
259/259 [==============================] - 0s 531us/step - loss: 0.3493 - accuracy: 0.8447
Epoch 83/100
259/259 [==============================] - 0s 539us/step - loss: 0.3372 - accuracy: 0.8539
Epoch 84/100
259/259 [==============================] - 0s 506us/step - loss: 0.3449 - accuracy: 0.8487
Epoch 85/100
259/259 [==============================] - 0s 519us/step - loss: 0.3429 - accuracy: 0.8465
Epoch 86/100
259/259 [==============================] - 0s 541us/step - loss: 0.3334 - accuracy: 0.8474
Epoch 87/100
259/259 [==============================] - 0s 546us/step - loss: 0.3444 - accuracy: 0.8455
Epoch 88/100
259/259 [==============================] - 0s 518us/step - loss:

# Ensemble Technique with Undersampling

In [106]:
xxdf.Churn.value_counts()

0    5163
1    1869
Name: Churn, dtype: int64

In [107]:
# Re-gain original features and labels

X = xxdf.drop('Churn', axis = 1)
y = xxdf['Churn']

In [108]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42, stratify = y)

In [109]:
y_train.value_counts()

0    4130
1    1495
Name: Churn, dtype: int64

In [110]:
df3 = X_train.copy()
df3['Churn'] = y_train

In [111]:
df3.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,OnlineSecurity,OnlineBackup,DeviceProtection,...,InternetService_Fiber optic,InternetService_No,Contract_Month-to-month,Contract_One year,Contract_Two year,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,Churn
1413,0,0,1,1,0.901408,1,1,1,1,1,...,1,0,0,0,1,0,1,0,0,0
7003,0,0,0,0,0.352113,0,0,0,0,1,...,0,0,1,0,0,0,0,1,0,0
3355,1,0,1,0,0.943662,1,1,0,1,1,...,1,0,0,0,1,0,1,0,0,0
4494,0,0,0,0,0.028169,1,0,0,1,0,...,1,0,1,0,0,0,0,1,0,0
3541,1,0,1,0,0.676056,0,0,1,0,0,...,0,0,1,0,0,1,0,0,0,0


In [112]:
df3_class0 = df3[df3.Churn == 0]
df3_class1 = df3[df3.Churn == 1]

In [117]:
(df3_class0.shape, df3_class1.shape)

((4130, 27), (1495, 27))

In [113]:
def get_train_batch(df_majority, df_minority, start, end):
    df_train = pd.concat([df_majority[start:end], df_minority], axis = 0)
    
    X_train = df_train.drop('Churn', axis = 'columns')
    y_train = df_train.Churn
    
    return X_train, y_train

In [114]:
X_train, y_train = get_train_batch(df3_class0, df3_class1, 0 , 1495)

y_pred1 = ANN(X_train, y_train, X_test, y_test,'binary_crossentropy', -1)

Epoch 1/100
94/94 [==============================] - 1s 627us/step - loss: 0.6289 - accuracy: 0.6416
Epoch 2/100
94/94 [==============================] - 0s 619us/step - loss: 0.5217 - accuracy: 0.7446
Epoch 3/100
94/94 [==============================] - 0s 560us/step - loss: 0.5053 - accuracy: 0.7472
Epoch 4/100
94/94 [==============================] - 0s 522us/step - loss: 0.4952 - accuracy: 0.7528
Epoch 5/100
94/94 [==============================] - 0s 605us/step - loss: 0.4790 - accuracy: 0.7672
Epoch 6/100
94/94 [==============================] - 0s 582us/step - loss: 0.4762 - accuracy: 0.7740
Epoch 7/100
94/94 [==============================] - 0s 575us/step - loss: 0.4822 - accuracy: 0.7685
Epoch 8/100
94/94 [==============================] - 0s 623us/step - loss: 0.4650 - accuracy: 0.7765
Epoch 9/100
94/94 [==============================] - 0s 558us/step - loss: 0.4731 - accuracy: 0.7776
Epoch 10/100
94/94 [==============================] - 0s 670us/step - loss: 0.4808 - accura

94/94 [==============================] - 0s 504us/step - loss: 0.3980 - accuracy: 0.8095
Epoch 82/100
94/94 [==============================] - 0s 525us/step - loss: 0.3906 - accuracy: 0.8216
Epoch 83/100
94/94 [==============================] - 0s 507us/step - loss: 0.3839 - accuracy: 0.8191
Epoch 84/100
94/94 [==============================] - 0s 514us/step - loss: 0.3868 - accuracy: 0.8261
Epoch 85/100
94/94 [==============================] - 0s 504us/step - loss: 0.3770 - accuracy: 0.8222
Epoch 86/100
94/94 [==============================] - 0s 551us/step - loss: 0.3753 - accuracy: 0.8299
Epoch 87/100
94/94 [==============================] - 0s 507us/step - loss: 0.3882 - accuracy: 0.8185
Epoch 88/100
94/94 [==============================] - 0s 500us/step - loss: 0.3631 - accuracy: 0.8279
Epoch 89/100
94/94 [==============================] - 0s 504us/step - loss: 0.3878 - accuracy: 0.8268
Epoch 90/100
94/94 [==============================] - 0s 525us/step - loss: 0.3805 - accuracy: 